$\Huge\color{Blue}{\text{Curator script for "La Diaria" - digital version}}$

$\large\color{brown}{\text{On how an open convention facilitated access to valuable data, and other things}}$

### To execute with graphic interface, just run **Curator()**

### To execute directly, use **search(word, section, mail, start_date, end_date)** ----- where:
### - **section** could be either: **Titular** or **Seccion** or **Comentario**
### - **mail** refers to the email address to be sent the report to
### - The **dates' format** is a tuple (YYYY,MM,DD)
### - Here an example: **search("saber", 'Titular', 'mail', (2018,12,17), (2020,6,30))**

$\LARGE\color{brown}{\text{Let's import libraries}}$


In [11]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup                                  #some of this libraries may require installation.
import requests
from datetime import datetime, timedelta
import glob
from pandas_profiling import ProfileReport
from smtplib import SMTP                                           #email outbound packages.
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table
import PySimpleGUI as sg        #GUI interface

pd.set_option('max_colwidth', 200)                                 #lenght of Comentario field requires this. 


$\LARGE\color{brown}{\text{Load the dataframe from csv file}}$

In [13]:
def convert(row):
#print(row)
   return '{}'.format(row['Link'],  row.name)

df=pd.read_csv('Proyecto La Diaria up to 2021-01-02.csv')         #load the DB from a file, either to work with or update
df.columns = ['Fecha', 'Titular', 'Seccion','Comentario', 'Link'] #load of the DB and some formating.
df['Link'] = df.apply(convert, axis=1)
df=df.drop_duplicates()

$\LARGE\color{brown}{\text{Let's define relevant functions to create the dates to scrap, and parse}}$

In [14]:
Today=(str(pd.to_datetime("today").date())[0:4], str(pd.to_datetime("today").date())[5:7], str(pd.to_datetime("today").date())[8:10])

def fechas(f1=(2014,9,8), f2=(2014,10,1)):                           #f1 and f2 in format (year, month, day)
    inicio = datetime(int(f1[0]),int(f1[1]),int(f1[2]))               #first edition for this newspaper was (2014,9,8)                                                                            
    fin    = datetime(int(f2[0]),int(f2[1]),int(f2[2]))     
    lista_fechas = [(inicio + timedelta(days=d)).strftime("%Y-%m-%d")
                    for d in range((fin - inicio).days + 1)] 
    return lista_fechas                   

def fechas_panda(lista_fechas):
    f=[]
    for i in lista_fechas:
        f.append(i.replace('-', '/'))
    return f

def armador_de_direcciones(lista_fechas):
    dates=[]
    direcciones=[]
    for i in lista_fechas:                                                  #creates the addresses to search
        year=i[0:4]
        month=i[5:7]
        day=i[8:10]
        dates = dates + [[year, month, day]]
    for j in dates:
        direcciones = direcciones + ['https://ladiaria.com.uy/edicion/{}/{}/{}/'.format(j[0], j[1], j[2])]    
    return direcciones

def convert(row):
#print(row)
   return '{}'.format(row['Link'],  row.name)

$\LARGE\color{brown}{\text{Parse functions}}$

In [15]:
  
def parse_element(card, selector):
    element=card.find('', selector)
    if element:
        return  " ".join(element.get_text().replace('\n', '').split())   #parse functions to extract what is needed.
    return ""
    
#titulo_foto= card.find('img')['title'] 

def parse_cards(date, list_cards, d):
    result=[]
    for card in list_cards:
        title=parse_element(card, {'class':'ld-card__title'})
        secc_auth=parse_element(card, {'class':'ld-card__byline'})
        coment=parse_element(card, {'class':'ld-card__intro'})
        result.append([date, title, secc_auth, coment, d])    
    return result

$\LARGE\color{brown}{\text{Let's define functions to filter according to the dates, word, and each section}}$

In [16]:
def construct_search_titular(word, fecha_incio=(2014,9,8), fecha_final=Today):
    Base=fechas_panda(fechas(fecha_incio, fecha_final))
    sift=df[df['Fecha'].isin(Base)]
    return sift[sift.Titular.str.contains(word,case=False)]          #construct and returns dataframes as results of the queries

def construct_search_section(word, fecha_incio=(2014,9,8), fecha_final=Today):
    Base=fechas_panda(fechas(fecha_incio, fecha_final))
    sift=df[df['Fecha'].isin(Base)]
    return sift[sift.Seccion.str.contains(word,case=False, na=False)]  

def construct_search_comment(word, fecha_incio=(2014,9,8), fecha_final=Today):
    Base=fechas_panda(fechas(fecha_incio, fecha_final))
    sift=df[df['Fecha'].isin(Base)]
    return sift[sift.Comentario.str.contains(word,case=False, na=False)]

$\LARGE\color{Blue}{\text{The Search function}}$


In [17]:
def search(word, section, mail, fecha_inicio=(2014,9,8), fecha_final=Today):            #search function takes values for 'section' as "Titular" or 'Seccion' or 'Comentario'
    if section=='Titular':
        send_table(construct_search_titular(word, fecha_inicio, fecha_final), word, mail, section, fecha_inicio, fecha_final)        #will send the email to the mail facilitated.
        return "Mail sent successfully."
    elif section=='Seccion':
        send_table(construct_search_section(word, fecha_inicio, fecha_final), word, mail, section, fecha_inicio, fecha_final)                                                              
        return "Mail sent successfully."
    elif section=='Comentario':
        send_table(construct_search_comment(word, fecha_inicio, fecha_final), word, mail, section, fecha_inicio, fecha_final)
        return "Mail sent successfully."

$\LARGE\color{brown}{\text{Functions to send the email as an outcome}}$

In [18]:
def send_mail(body, mail, word, section):
    message = MIMEMultipart()
    message['Subject'] = 'Query results: -{}- in -{}-'.format(word, section)                                                
    message['From'] = 'mail'         #shhould be the email to send the mails from
    message['To'] = str(mail)  
    body_content = body
    message.attach(MIMEText(body_content, "html"))
    msg_body = message.as_string()

    server = SMTP('smtp.gmail.com', 587)       
    server.starttls()            #It is needeed to allow less secure apps on the gmail settings for this to work.
    server.login(message['From'], 'password')   #should be the password of the email to send the mails from
    server.sendmail(message['From'], message['To'], msg_body)
    server.quit()

def send_table(table, word, mail, section, fecha_inicio=(2014,9,8),fecha_final=Today):
    output = 'Please see below results for your query: {} in {} period: {}, {}'.format(word, section, fecha_inicio, fecha_final) + build_table(table, 'blue_light') + "All results informed for '{}' in '{}'' ".format(word, section)
    send_mail(output, mail, word, section)
    return "Mail sent successfully."

$\normalsize\color{black}{\text{Scrapping code, only execute if the dates has been updated for scrapping}}$

In [9]:
# #this code creates the dataframe and scrap fopr the dates facilitated above. 
#lista_fechas=fechas((2020,12,19), (2021,1,2))           #these parameters need to be facilitated to scrap the timeframe needed.                                          
#direcciones=armador_de_direcciones(lista_fechas)   
# master=[]
# final=[]
# h=0
# master=[]
# for d in direcciones:
#      date=str(d[32:42])
#      address= requests.get(d)
#      soup = BeautifulSoup(address.text, 'html.parser')
#      section=soup.find('section', {'class':'ld-grid-main'})     
#      try:
#          list_cards=section.find_all('div', {'class':'ld-card'})                                          
#          master.append(parse_cards(date, list_cards,d))
#      except:
#          h=h+1

# for i in master:
#      for j in i:
#          final.append(j)
# df2=pd.DataFrame(final)
# df2.columns = ['Fecha', 'Titular', 'Seccion','Comentario', 'Link'] #load of the DB and some formating.
# df2['Link'] = df2.apply(convert, axis=1)
# df2=df2.drop_duplicates()
#df2=df2.iloc[::-1]
#df2.reset_index()

$\normalsize\color{black}{\text{Update the dataframe. Only execute if needed}}$


In [28]:
#result = pd.concat([df2,df])
#result.to_csv('Proyecto La Diaria up to {}.csv'.format(fechas[-1]), index=False)

$\LARGE\color{Blue}{\text{Curator() main function to use the search with graphic interface}}$


In [43]:
##Note: if the date field is left blank will create an error
def Curator():
    sg.theme('SandyBeach')      
    fecha_inicio=[]
    fecha_final=[]
    layout = [ 
        [sg.Text('Please enter the dates to look at, the word to search, the section (Titular, Seccion o Comentario) and the email to receive the report.')], 
        
        [sg.Text('Start Date in format DD/MM/YYYY', size =(27, 1)), sg.InputText('8/9/2014')],
        [sg.Text('End Date in format DD/MM/YYYY', size =(27, 1)), sg.InputText('{}/{}/{}'.format(Today[2], Today[1], Today[0]))],
        [sg.Text('Word', size =(15, 1)), sg.InputText()], 
        [sg.Text('Section', size =(15, 1)), sg.InputText()], 
        [sg.Text('Email', size =(15, 1)), sg.InputText()], 
        [sg.Submit(), sg.Cancel()] 
    ] 
  
    window = sg.Window('La Diaria - Curator', layout) 
    event, values = window.read() 
    window.close() 
    fi=values[0].split('/')[::-1]
    ff=values[1].split('/')[::-1]
    for i in fi:
        fecha_inicio.append(int(i))
    fecha_inicio=tuple(fecha_inicio)
    for i in ff:
        fecha_final.append(int(i))
    fecha_final=tuple(fecha_final)  
                   
    search(values[2], values[3], values[4], fecha_inicio, fecha_final)
    return "Mail Sent"

$\LARGE\color{Blue}{\text{Example of search with dates, if not provided, will be searched in all the dataframe}}$

In [46]:
search("dormir", 'Titular', 'mail', (2018,12,17), (2020,6,30)) #mail should be the one to be sent the report to.


'Mail sent successfully.'

In [48]:
search("dormir", 'Titular', 'mail')  #Search in all dataframe  #mail should be the one to be sent the report to.

'Mail sent successfully.'

$\LARGE\color{Blue}{\text{To Run the Graphic interface, execute Curator()}}$

In [ ]:
Curator()